# Guidelines for ETL Project

This document contains guidelines, requirements, and suggestions for Project 1.

## Team Effort

Due to the short timeline, teamwork will be crucial to the success of this project! Work closely with your team through all phases of the project to ensure that there are no surprises at the end of the week.

Working in a group enables you to tackle more difficult problems than you'd be able to working alone. In other words, working in a group allows you to **work smart** and **dream big**. Take advantage of it!

## Project Proposal

Before you start writing any code, remember that you only have one week to complete this project. View this project as a typical assignment from work. Imagine a bunch of data came in and you and your team are tasked with migrating it to a production data base.

Take advantage of your Instructor and TA support during office hours and class project work time. They are a valuable resource and can help you stay on track.

## Finding Data

Your project must use 2 or more sources of data. We recommend the following sites to use as sources of data:

* [data.world](https://data.world/)

* [Kaggle](https://www.kaggle.com/)

You can also use APIs or data scraped from the web. However, get approval from your instructor first. Again, there is only a week to complete this!

## Data Cleanup & Analysis

Once you have identified your datasets, perform ETL on the data. Make sure to plan and document the following:

* The sources of data that you will extract from.

* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).

* The type of final production database to load the data into (relational or non-relational).

* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report

At the end of the week, your team will submit a Final Report that describes the following:

* **E**xtract: your original data sources and how the data was formatted (CSV, JSON, pgAdmin 4, etc).

* **T**ransform: what data cleaning or transformation was required.

* **L**oad: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

- - -

### Copyright

Coding Boot Camp © 2019. All Rights Reserved.


In [2]:
! pip install python-dotenv

In [3]:
! pip install psycopg2

In [4]:
! pip install yelpapi

In [38]:
# Import dependencies

import pandas as pd
from pymongo import MongoClient
import pymongo

from yelpapi import YelpAPI
import requests
from config import yelp_key
from pprint import pprint
import pandas as pd
import json

import collections
import re, string
import sys
import time


In [5]:

# create a yelp api instance
yelp_api = YelpAPI(yelp_key)

print(yelp_api)

### Store CSV into DataFrame

In [44]:
zip_zhvi_single_family_homes_time_series_file = "./Resources/Zip_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_single_family_homes_time_series_df = pd.read_csv(zip_zhvi_single_family_homes_time_series_file)

# Add in Home-Type Column
zip_zhvi_single_family_homes_time_series_df["Home Type"] = "Single Family Home"

# Visualize
print(f"Count of Single Family Home: {len(zip_zhvi_single_family_homes_time_series_df)}")

zip_zhvi_single_family_homes_time_series_df.head()

### Store CSV into DataFrame

Count of Single Family Home: 30342


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,960802.0,962396.0,963537.0,960187.0,955918.0,949634.0,948112.0,948911.0,954998.0,Single Family Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,298619.0,...,793957.0,795186.0,796847.0,797731.0,798012.0,798051.0,799653.0,801950.0,806333.0,Single Family Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1423693.0,1424519.0,1420952.0,1414127.0,1407642.0,1407057.0,1404420.0,1402979.0,1402128.0,Single Family Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,204122.0,...,342174.0,342397.0,342883.0,343979.0,344922.0,345504.0,345781.0,346558.0,347253.0,Single Family Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,456771.0,...,999819.0,1001592.0,1003223.0,1004383.0,1004370.0,1005620.0,1008258.0,1011170.0,1016459.0,Single Family Home


In [7]:
zip_zhvi_condo_coop_time_series_file = "./Resources/Zip_zhvi_uc_condo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_condo_coop_time_series_df = pd.read_csv(zip_zhvi_condo_coop_time_series_file)

# Add in Home-Type Column
zip_zhvi_condo_coop_time_series_df["Home Type"] = "Condos and Co-op Home"

# Covert to dictionary to load into Mongo DB
zip_zhvi_condo_coop_time_series_dict = zip_zhvi_condo_coop_time_series_df.to_dict("records")

# Visualize
print(len(zip_zhvi_condo_coop_time_series_df))
zip_zhvi_condo_coop_time_series_df.head()


10664


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,187153.0,...,997141.0,976952.0,952332.0,941857.0,934084.0,931789.0,930503.0,931641.0,935926.0,Condos and Co-op Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,194388.0,...,398247.0,399194.0,400544.0,401578.0,401754.0,402238.0,403466.0,405361.0,407901.0,Condos and Co-op Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,236945.0,...,1055892.0,1068214.0,1068853.0,1078253.0,1083853.0,1089299.0,1092674.0,1100314.0,1117604.0,Condos and Co-op Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,NaN,...,247259.0,246484.0,245794.0,245984.0,246056.0,246320.0,246270.0,246636.0,246470.0,Condos and Co-op Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,202096.0,...,459621.0,460100.0,461202.0,461424.0,461303.0,461430.0,462691.0,464602.0,467196.0,Condos and Co-op Home


In [8]:

zip_zhvi_one_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_1_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_one_bedroom_time_series_df = pd.read_csv(zip_zhvi_one_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_one_bedroom_time_series_df["Home Type"] = "One Bedroom Home"

# Covert to dictionary to load into Mongo DB
zip_zhvi_one_bedroom_time_series_dict = zip_zhvi_one_bedroom_time_series_df.to_dict("records")

# Visualize
print(len(zip_zhvi_one_bedroom_time_series_df))
zip_zhvi_one_bedroom_time_series_df.head()


18560


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,102766.0,...,657379.0,657478.0,658265.0,657925.0,656708.0,655825.0,654157.0,651796.0,648394.0,One Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,117621.0,...,234361.0,234935.0,235555.0,236073.0,236282.0,236818.0,237992.0,239587.0,241320.0,One Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,158083.0,...,679903.0,678316.0,677287.0,675021.0,673560.0,673517.0,672950.0,672538.0,672618.0,One Bedroom Home
3,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,133907.0,...,265891.0,265954.0,266367.0,266759.0,267050.0,267434.0,267815.0,267932.0,267699.0,One Bedroom Home
4,91940,5,77449,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,79426.0,...,126198.0,126739.0,127296.0,127984.0,128896.0,129507.0,130055.0,130456.0,131683.0,One Bedroom Home


In [9]:

zip_zhvi_two_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_2_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_two_bedroom_time_series_df = pd.read_csv(zip_zhvi_two_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_two_bedroom_time_series_df["Home Type"] = "Two Bedroom Home"


# Covert to dictionary to load into Mongo DB
zip_zhvi_two_bedroom_time_series_dict = zip_zhvi_two_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_two_bedroom_time_series_df))
zip_zhvi_two_bedroom_time_series_df.head()


26681


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,211516.0,...,1115816.0,1115066.0,1112470.0,1110355.0,1106770.0,1104648.0,1103091.0,1101730.0,1100391.0,Two Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,199947.0,...,411315.0,412108.0,413346.0,414344.0,414549.0,415063.0,416473.0,418371.0,420872.0,Two Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1721495.0,1718819.0,1715486.0,1711585.0,1707480.0,1705946.0,1701921.0,1698954.0,1699372.0,Two Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,NaN,...,269660.0,269406.0,269428.0,270495.0,271643.0,272388.0,272513.0,273061.0,273637.0,Two Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,217292.0,...,467846.0,468056.0,468974.0,469130.0,469143.0,469344.0,470710.0,472715.0,475446.0,Two Bedroom Home


In [10]:

zip_zhvi_three_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_3_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_three_bedroom_time_series_df = pd.read_csv(zip_zhvi_three_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_three_bedroom_time_series_df["Home Type"] = "Three Bedroom Home"

# Covert to dictionary to load into Mongo DB
zip_zhvi_three_bedroom_time_series_dict = zip_zhvi_three_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_three_bedroom_time_series_df))
zip_zhvi_three_bedroom_time_series_df.head()


28728


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,992309.0,...,1929377.0,1919420.0,1910535.0,1904880.0,1901163.0,1901511.0,1902212.0,1907083.0,1920389.0,Three Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,249507.0,...,579817.0,581079.0,583177.0,584626.0,585485.0,586645.0,588865.0,592509.0,597737.0,Three Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,2672494.0,2663477.0,2661284.0,2646807.0,2638973.0,2628182.0,2605343.0,2573719.0,2542950.0,Three Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,144763.0,...,266793.0,266797.0,267130.0,268143.0,269206.0,269880.0,270129.0,270714.0,271267.0,Three Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,313324.0,...,687148.0,687108.0,687736.0,687137.0,686859.0,686394.0,687508.0,689081.0,693130.0,Three Bedroom Home


In [11]:

zip_zhvi_four_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_4_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_four_bedroom_time_series_df = pd.read_csv(zip_zhvi_four_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_four_bedroom_time_series_df["Home Type"] = "Four Bedroom Home"


# Covert to dictionary to load into Mongo DB
zip_zhvi_four_bedroom_time_series_dict = zip_zhvi_four_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_four_bedroom_time_series_df))
zip_zhvi_four_bedroom_time_series_df.head()


26467


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,2408740.0,2409014.0,2404003.0,2391342.0,2388913.0,2398323.0,2411226.0,2428734.0,2455802.0,Four Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,366999.0,...,921170.0,921419.0,922116.0,922445.0,922196.0,922128.0,924016.0,927506.0,933594.0,Four Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,5955760.0,5902502.0,5836962.0,5796591.0,5736771.0,5705849.0,5656754.0,5598082.0,5511871.0,Four Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,195492.0,...,343105.0,343293.0,343752.0,344796.0,345678.0,346212.0,346470.0,347230.0,347967.0,Four Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,471019.0,...,1134482.0,1137006.0,1139337.0,1140929.0,1140258.0,1141648.0,1145064.0,1148874.0,1155107.0,Four Bedroom Home


In [12]:
# 5 and 5 plus bedroom homes
zip_zhvi_five_and_five_plus_bedroom_time_series_file = "./Resources/Zip_zhvi_bdrmcnt_5_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv"
zip_zhvi_five_and_five_plus_bedroom_time_series_df = pd.read_csv(zip_zhvi_five_and_five_plus_bedroom_time_series_file)

# Add in Home-Type Column
zip_zhvi_five_and_five_plus_bedroom_time_series_df["Home Type"] = "Five and Five Plus Bedroom Home"


# Covert to dictionary to load into Mongo DB
zip_zhvi_five_and_five_plus_bedroom_time_series_dict = zip_zhvi_five_and_five_plus_bedroom_time_series_df.to_dict("records")


# Visualize
print(len(zip_zhvi_five_and_five_plus_bedroom_time_series_df))
zip_zhvi_five_and_five_plus_bedroom_time_series_df.head()


21871


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,3526803.0,3588803.0,3644455.0,3658205.0,3682981.0,3758156.0,3896928.0,4024298.0,4116546.0,Five and Five Plus Bedroom Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,NaN,...,1448109.0,1450282.0,1453442.0,1455531.0,1457204.0,1459856.0,1465304.0,1471079.0,1480683.0,Five and Five Plus Bedroom Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,8831883.0,8756920.0,8598371.0,8426361.0,8327894.0,8333295.0,8323928.0,8329391.0,8286910.0,Five and Five Plus Bedroom Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,240085.0,...,447991.0,448243.0,449130.0,450705.0,452194.0,453019.0,453588.0,454693.0,455474.0,Five and Five Plus Bedroom Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,699792.0,...,1833778.0,1837304.0,1839939.0,1842239.0,1843137.0,1848439.0,1857536.0,1866711.0,1881082.0,Five and Five Plus Bedroom Home


In [35]:
# Combined data frame
combined_zip_zhvi_homes_time_series_df = pd.concat([zip_zhvi_single_family_homes_time_series_df,
                                               zip_zhvi_condo_coop_time_series_df,
                                               zip_zhvi_one_bedroom_time_series_df,
                                               zip_zhvi_two_bedroom_time_series_df,
                                               zip_zhvi_three_bedroom_time_series_df,
                                               zip_zhvi_four_bedroom_time_series_df,
                                               zip_zhvi_five_and_five_plus_bedroom_time_series_df
                                              ], ignore_index=True)

# Visualize
print(len(combined_zip_zhvi_homes_time_series_df))
combined_zip_zhvi_homes_time_series_df.head()


163313


,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,960802.0,962396.0,963537.0,960187.0,955918.0,949634.0,948112.0,948911.0,954998.0,Single Family Home
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,298619.0,...,793957.0,795186.0,796847.0,797731.0,798012.0,798051.0,799653.0,801950.0,806333.0,Single Family Home
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,NaN,...,1423693.0,1424519.0,1420952.0,1414127.0,1407642.0,1407057.0,1404420.0,1402979.0,1402128.0,Single Family Home
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,204122.0,...,342174.0,342397.0,342883.0,343979.0,344922.0,345504.0,345781.0,346558.0,347253.0,Single Family Home
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,456771.0,...,999819.0,1001592.0,1003223.0,1004383.0,1004370.0,1005620.0,1008258.0,1011170.0,1016459.0,Single Family Home


### Clean DataFrame

In [36]:
# Drop columns Region ID, Sizerank, StateName
drop_columns = ['RegionID', 'SizeRank', 'StateName']

combined_zip_zhvi_homes_time_series_df.drop(drop_columns, inplace=True, axis=1)


In [40]:
%%time

# Rename columns
combined_zip_zhvi_homes_time_series_df.rename(columns={'RegionName': 'ZipCode'}, inplace=True)

# Clean NaN Values by replacing NaN with 'na'
combined_zip_zhvi_homes_time_series_df = combined_zip_zhvi_homes_time_series_df.fillna('na')

combined_zip_zhvi_homes_time_series_df["Price Trend Flag"] = "No Trend"

# Calculate the Price trend flag

previous_latest_month_end_date = '2020-08-31'
latest_month_end_date = '2020-09-30'

for index, row in combined_zip_zhvi_homes_time_series_df.iterrows():
    
    if row[latest_month_end_date] < row[previous_latest_month_end_date]:
        combined_zip_zhvi_homes_time_series_df.loc[index,"Price Trend Flag"] = "Down Trend"
        
    elif row[latest_month_end_date] > row[previous_latest_month_end_date]:
        combined_zip_zhvi_homes_time_series_df.loc[index,"Price Trend Flag"] = "Up Trend"

# Covert to dictionary to load into Mongo DB
combined_zip_zhvi_homes_time_series_dict = combined_zip_zhvi_homes_time_series_df.to_dict("records")

# Display cleaned dataframe
combined_zip_zhvi_homes_time_series_df.head()


Wall time: 35min 48s


,ZipCode,RegionType,State,City,Metro,CountyName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,...,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,Home Type,Price Trend Flag
0,10025,Zip,NY,New York,New York-Newark-Jersey City,New York County,na,na,na,na,...,962396.0,963537.0,960187.0,955918.0,949634.0,948112.0,948911.0,954998.0,Single Family Home,Up Trend
1,60657,Zip,IL,Chicago,Chicago-Naperville-Elgin,Cook County,298619,298021,297566,297041,...,795186.0,796847.0,797731.0,798012.0,798051.0,799653.0,801950.0,806333.0,Single Family Home,Up Trend
2,10023,Zip,NY,New York,New York-Newark-Jersey City,New York County,na,na,na,na,...,1424519.0,1420952.0,1414127.0,1407642.0,1407057.0,1404420.0,1402979.0,1402128.0,Single Family Home,Down Trend
3,77494,Zip,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,204122,204375,204174,202964,...,342397.0,342883.0,343979.0,344922.0,345504.0,345781.0,346558.0,347253.0,Single Family Home,Up Trend
4,60614,Zip,IL,Chicago,Chicago-Naperville-Elgin,Cook County,456771,456410,455758,455708,...,1001592.0,1003223.0,1004383.0,1004370.0,1005620.0,1008258.0,1011170.0,1016459.0,Single Family Home,Up Trend


In [39]:
combined_zip_zhvi_homes_time_series_df.count()

ZipCode             163313
RegionType          163313
State               163313
City                163313
Metro               163313
                     ...  
2020-07-31          163313
2020-08-31          163313
2020-09-30          163313
Home Type           163313
Price Trend Flag    163313
Length: 305, dtype: int64

### Connect to local database

In [45]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and homes collection
db = client.data_etl_db
homes_collection = db.homes

In [46]:
# Load the collection
homes_collection.delete_many({})

#data.reset_index(inplace=True)
#data_dict = data.to_dict("records")
# Insert collection
homes_collection.insert_many(combined_zip_zhvi_homes_time_series_dict)


In [52]:
query_homes_collection = homes_collection.aggregate([
    ## stage 1
    {
        "$match" : 
                 {"Price Trend Flag" : {"$eq" : "Down Trend" },
                  "Home Type": {"$eq": "Single Family Home"},
                  "City": {"$eq": "San Francisco"}}
    },
    ## stage 2
    {
        "$count" : "total_rows"
    }
])

for i in query_homes_collection:
    print(i)

print(f"There is a Down Trend in {i['total_rows']} ZipCodes in Single Family Homes")

{'total_rows': 12}
There is a Down Trend in 12 ZipCodes in Single Family Homes


### Create new data with select columns

In [65]:
# Fill in parameters for term, location and search_limit
term = 'Grocery'
location = 'San Francisco, CA'
search_limit = 50
category = 'Grocery (grocery)'
# Create the response
try:
    response = yelp_api.search_query(term=term, location = location, category=category)
except:
    print("Yelp API Error!")
    pass

# Check data type of response
type(response)

dict

In [66]:
pprint(f"{response['businesses'][0]}")

("{'id': 'TzY3Z39P5l9_LIrG-PcUng', 'alias': 'mollie-stones-san-francisco-5', "
 '\'name\': "Mollie Stone\'s", \'image_url\': '
 "'https://s3-media4.fl.yelpcdn.com/bphoto/37vIMFfjeWEOgavS-N0h0w/o.jpg', "
 "'is_closed': False, 'url': "
 "'https://www.yelp.com/biz/mollie-stones-san-francisco-5?adjust_creative=5AykeSTc_WsQao1ge5PMQg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=5AykeSTc_WsQao1ge5PMQg', "
 "'review_count': 154, 'categories': [{'alias': 'grocery', 'title': "
 "'Grocery'}, {'alias': 'markets', 'title': 'Fruits & Veggies'}], 'rating': "
 "3.5, 'coordinates': {'latitude': 37.76068, 'longitude': -122.43639}, "
 "'transactions': [], 'price': '$$$', 'location': {'address1': '4201 18th St', "
 "'address2': '', 'address3': '', 'city': 'San Francisco', 'zip_code': "
 "'94114', 'country': 'US', 'state': 'CA', 'display_address': ['4201 18th St', "
 "'San Francisco, CA 94114']}, 'phone': '+14152558959', 'display_phone': "
 "'(415) 255-8959', 'distance': 33.974786

In [67]:
pprint(response)

{'businesses': [{'alias': 'mollie-stones-san-francisco-5',
                 'categories': [{'alias': 'grocery', 'title': 'Grocery'},
                                {'alias': 'markets',
                                 'title': 'Fruits & Veggies'}],
                 'coordinates': {'latitude': 37.76068, 'longitude': -122.43639},
                 'display_phone': '(415) 255-8959',
                 'distance': 33.97478673903397,
                 'id': 'TzY3Z39P5l9_LIrG-PcUng',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/37vIMFfjeWEOgavS-N0h0w/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '4201 18th St',
                              'address2': '',
                              'address3': '',
                              'city': 'San Francisco',
                              'country': 'US',
                              'display_address': ['4201 18th St',
                                                  'San Francisc

In [68]:
# Check key of response dictionary (Alternative to scroll through text to look for keys)
print(response.keys())

dict_keys(['businesses', 'total', 'region'])


In [69]:
# Set list of keys as column names
cols = list(response['businesses'][0].keys())

# Ceate a DataFrame
yelp_df = pd.DataFrame(columns=cols)

for biz in response['businesses']:
    yelp_df = yelp_df.append(biz, ignore_index=True)

yelp_dict = yelp_df.to_dict("records")
    
yelp_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,TzY3Z39P5l9_LIrG-PcUng,mollie-stones-san-francisco-5,Mollie Stone's,https://s3-media4.fl.yelpcdn.com/bphoto/37vIMF...,False,https://www.yelp.com/biz/mollie-stones-san-fra...,154,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.5,"{'latitude': 37.76068, 'longitude': -122.43639}",[],$$$,"{'address1': '4201 18th St', 'address2': '', '...",+14152558959,(415) 255-8959,33.974787
1,GOBChLG2VZDk2nK-RftPIA,richmond-new-may-wah-supermarket-san-francisco,Richmond New May Wah Supermarket,https://s3-media2.fl.yelpcdn.com/bphoto/qA4kTQ...,False,https://www.yelp.com/biz/richmond-new-may-wah-...,451,"[{'alias': 'intlgrocery', 'title': 'Internatio...",4.0,"{'latitude': 37.78258, 'longitude': -122.46685}",[],$,"{'address1': '707-719 Clement St', 'address2':...",+14152219826,(415) 221-9826,3603.070973
2,YfCDZdvVnqw1n_qP6U9BOQ,trader-joes-san-francisco,Trader Joe's,https://s3-media3.fl.yelpcdn.com/bphoto/UhD7wv...,False,https://www.yelp.com/biz/trader-joes-san-franc...,556,"[{'alias': 'grocery', 'title': 'Grocery'}]",4.0,"{'latitude': 37.7833841121786, 'longitude': -1...",[],$$,"{'address1': '3 Masonic Ave', 'address2': '', ...",+14153469964,(415) 346-9964,2693.669066
3,SIoBlo_eCp0TRpeg9rPfdw,safeway-san-francisco-13,Safeway,https://s3-media2.fl.yelpcdn.com/bphoto/tbyxBq...,False,https://www.yelp.com/biz/safeway-san-francisco...,334,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 37.7746023, 'longitude': -122.465...",[],$$,"{'address1': '735 7th Ave', 'address2': '', 'a...",+14156834070,(415) 683-4070,2951.969685
4,C0xAUgp3KHIFyjIYg-GqRA,grocery-outlet-bargain-market-san-francisco-2,Grocery Outlet Bargain Market,https://s3-media4.fl.yelpcdn.com/bphoto/aBlVcM...,False,https://www.yelp.com/biz/grocery-outlet-bargai...,164,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",4.0,"{'latitude': 37.7796711, 'longitude': -122.487...",[],$,"{'address1': '6333 Geary Blvd', 'address2': ''...",+14158765576,(415) 876-5576,4956.942003


In [70]:
# Create new dataframe with [id, name, url, review_count, rating, transactions and price]
yelp_new_df = yelp_df[['name', 'url', 'review_count', 'rating', 'transactions','price']]

yelp_new_dict = yelp_new_df.to_dict("records")

In [71]:
yelp_new_df.head()

,name,url,review_count,rating,transactions,price
0,Mollie Stone's,https://www.yelp.com/biz/mollie-stones-san-fra...,154,3.5,[],$$$
1,Richmond New May Wah Supermarket,https://www.yelp.com/biz/richmond-new-may-wah-...,451,4.0,[],$
2,Trader Joe's,https://www.yelp.com/biz/trader-joes-san-franc...,556,4.0,[],$$
3,Safeway,https://www.yelp.com/biz/safeway-san-francisco...,334,2.5,[],$$
4,Grocery Outlet Bargain Market,https://www.yelp.com/biz/grocery-outlet-bargai...,164,4.0,[],$


In [72]:
yelp_new_df.count()

name            20
url             20
review_count    20
rating          20
transactions    20
price           20
dtype: int64

### Connect to local database

In [73]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

yelp_db = client.data_etl_db
yelp_collection = yelp_db.yelp_api


In [74]:
# Define Yelp collection
yelp_collection = yelp_db.yelp_api

In [75]:
# Load the collection
yelp_collection.delete_many({})

#data.reset_index(inplace=True)
#data_dict = data.to_dict("records")
# Insert collection
yelp_collection.insert_many(yelp_new_dict)



In [87]:
query_yelp_api_collection = yelp_collection.find({
    "rating" : {  "$gt" : 3.0, "$lt" : 5.0  }
})

for i in query_yelp_api_collection:
    print(f"Business Name: {i['name']}")


Business Name: Mollie Stone's
Business Name: Richmond New May Wah Supermarket
Business Name: Trader Joe's
Business Name: Grocery Outlet Bargain Market
Business Name: Falletti Foods
Business Name: 22nd & Irving Market
Business Name: Trader Joe's
Business Name: Nijiya Market - San Francisco
Business Name: Buffalo Whole Food & Grain Company
Business Name: Bryan's Grocery
Business Name: Super Mira Market
Business Name: Haight Street Market
Business Name: Christa Wonderful Market


### Store JSON data into a DataFrame

In [88]:
def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file, encoding="cp437", errors='ignore') as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count ==0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)

In [22]:
yelp_academic_dataset_business_json_file = "Resources/yelp_academic_dataset_business.json"
#yelp_academic_dataset_business_df = pd.read_json(yelp_academic_dataset_business_json_file)
#yelp_academic_dataset_business_df.head()

In [89]:
%%time

yelp_academic_dataset_business_data_func = read_json(yelp_academic_dataset_business_json_file)

Wall time: 11.1 s


In [90]:
yelp_academic_dataset_business_data_func.count()

business_id     209393
name            209393
address         209393
city            209393
state           209393
postal_code     209393
latitude        209393
longitude       209393
stars           209393
review_count    209393
is_open         209393
attributes      180348
categories      208869
hours           164550
dtype: int64

In [91]:
%%time

# read the entire file into a python array
with open(yelp_academic_dataset_business_json_file, encoding="cp437", errors='ignore') as f:
    yelp_academic_dataset_business_data = f.readlines()

# remove the trailing "\n" from each line
yelp_academic_dataset_business_data = map(lambda x: x.rstrip(), yelp_academic_dataset_business_data)

yelp_academic_dataset_business_data_json_str = "[" + ','.join(yelp_academic_dataset_business_data) + "]"

# now, load it into pandas
yelp_academic_dataset_business_data_df = pd.read_json(yelp_academic_dataset_business_data_json_str)



Wall time: 15.4 s


In [92]:
yelp_academic_dataset_business_data_dict = yelp_academic_dataset_business_data_df.to_dict("records")
#yelp_academic_dataset_business_data_df.head()

In [93]:
yelp_academic_dataset_business_data_df.count()

business_id     209393
name            209393
address         209393
city            209393
state           209393
postal_code     209393
latitude        209393
longitude       209393
stars           209393
review_count    209393
is_open         209393
attributes      180348
categories      208869
hours           164550
dtype: int64

In [94]:
yelp_academic_dataset_business_data_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


In [96]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

yelp_db = client.data_etl_db
yelp_business_collection = yelp_db.yelp_business

In [153]:
# Load the collection
yelp_business_collection.delete_many({})

#data.reset_index(inplace=True)
#data_dict = data.to_dict("records")
# Insert collection
yelp_business_collection.insert_many(yelp_academic_dataset_business_data_dict)


In [170]:
yelp_count_result = yelp_academic_dataset_business_data_df.loc[yelp_academic_dataset_business_data_df["state"] == "CA"].count()

print(f"The count of business in california: {yelp_count_result}")

The count of business in california: business_id     23
name            23
address         23
city            23
state           23
postal_code     23
latitude        23
longitude       23
stars           23
review_count    23
is_open         23
attributes      21
categories      23
hours           17
dtype: int64
